In [168]:
import os
from pprint import pprint
import re
import random
import pandas as pd
import numpy as np
import transformers
from konlpy.tag import Okt

## 데이터 읽어오기

In [2]:
data_df = pd.read_csv("../data/train/train.tsv", sep="\t", header=None)
data_df

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님
...,...,...,...,...,...,...,...,...,...
8995,wikipedia-5414-12-15-21-0-4,2002년 FIFA 월드컵 사우디아라비아와의 1차전에서 독일은 8-0으로 승리하였는...,사우디아라비아,15,21,2002년,0,4,관계_없음
8996,wikipedia-10384-4-12-14-0-1,일본의 2대 메이커인 토요타와 닛산은 시장 점유율을 높이기 위한 신차 개발을 계속하...,토요타,12,14,일본,0,1,단체:본사_국가
8997,wikipedia-25913-6-8-10-93-106,방호의의 손자 방덕룡(方德龍)은 1588년(선조 21년) 무과에 급제하고 낙안군수로...,방덕룡,8,10,선무원종공신(宣武原從功臣),93,106,인물:직업/직함
8998,wikitree-12062-15-0-3-46-47,LG전자는 올해 초 국내시장에 출시한 2020년형 ‘LG 그램’ 시리즈를 이달부터 ...,LG전자,0,3,북미,46,47,관계_없음


## 모델 생성하기

In [77]:
model_name = "monologg/koelectra-base-v3-discriminator"
# model_name = "bert-base-multilingual-cased"
model = transformers.ElectraModel.from_pretrained(model_name)

## 토크나이저 생성하기

In [25]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

## 파이프라인 생성하기

In [31]:
model_name = "monologg/koelectra-base-v3-generator"
mask_filler = transformers.pipeline('fill-mask', model=model_name)

Downloading:   0%|          | 0.00/463 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/149M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

## 테스트

In [160]:
sentence = "영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버 ( Land Rover ) 와 지프 ( Jeep ) 가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다."

In [169]:
okt = Okt()

In [171]:
okt.phrases(sentence)

['영국',
 '사용',
 '사용되는 스포츠',
 '사용되는 스포츠 유틸리티',
 '사용되는 스포츠 유틸리티 자동차',
 '사용되는 스포츠 유틸리티 자동차의 브랜드',
 '랜드로버',
 'Land',
 'Land Rover',
 '지프',
 'Jeep',
 '이 브랜드들',
 '자동차',
 '자동차의 종류',
 '말로',
 '말로 사용',
 '기도',
 '스포츠',
 '유틸리티',
 '브랜드',
 'Rover',
 '브랜드들',
 '종류']

In [178]:
tokenized_sentence = tokenizer.tokenize(sentence)
print(sentence)
print(tokenized_sentence)
print(tokenizer.convert_tokens_to_string(tokenized_sentence))

1950년 9월 28일, 수복 후 일부 장관들의 경질에 따라 법무부 장관에 김준연, 농림부 장관에 공진항, 사회부 장관에 허정, 문교부 장관에 백낙준, 보건부 장관에 오한영이 임명되었다.
['1950', '##년', '9', '##월', '28', '##일', ',', '수복', '후', '일부', '장관', '##들의', '경질', '##에', '따라', '법무부', '장관', '##에', '김준', '##연', ',', '농림', '##부', '장관', '##에', '공', '##진', '##항', ',', '사회', '##부', '장관', '##에', '허정', ',', '문', '##교', '##부', '장관', '##에', '백', '##낙', '##준', ',', '보건', '##부', '장관', '##에', '오', '##한', '##영', '##이', '임명', '##되', '##었', '##다', '.']
1950년 9월 28일, 수복 후 일부 장관들의 경질에 따라 법무부 장관에 김준연, 농림부 장관에 공진항, 사회부 장관에 허정, 문교부 장관에 백낙준, 보건부 장관에 오한영이 임명되었다.


In [159]:
# [MASK]
mask_filler(sentence)

[{'sequence': '[CLS] 영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버 ( Land Rover ) 와 지프 ( Jeep ) 가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다. [SEP]',
  'score': 0.7162365317344666,
  'token': 3170,
  'token_str': '와'},
 {'sequence': '[CLS] 영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버 ( Land Rover ), 지프 ( Jeep ) 가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다. [SEP]',
  'score': 0.12225419282913208,
  'token': 16,
  'token_str': ','},
 {'sequence': '[CLS] 영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버 ( Land Rover ) 나 지프 ( Jeep ) 가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다. [SEP]',
  'score': 0.10778795182704926,
  'token': 2236,
  'token_str': '나'},
 {'sequence': '[CLS] 영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버 ( Land Rover ) 또는 지프 ( Jeep ) 가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다. [SEP]',
  'score': 0.02385668270289898,
  'token': 6587,
  'token_str': '또는'},
 {'sequence': '[CLS] 영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버 ( Land Rover ) 및 지프 ( Jeep ) 가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다. [SEP]',
  'score': 0.007861257530748844,
  'token': 2728,
  

In [189]:
rand_idx = random.randrange(len(data_df))
# rand_idx = 1
rand_data = data_df.iloc[rand_idx, :]

# ========================================
source = rand_data[0]
sentence = rand_data[1]

e1_name = rand_data[2]
e1_si = rand_data[3] - sentence[:rand_data[3]].count(" ")
e1_ei = rand_data[4] - sentence[:rand_data[4]].count(" ")

e2_name = rand_data[5]
e2_si = rand_data[6] - sentence[:rand_data[6]].count(" ")
e2_ei = rand_data[7] - sentence[:rand_data[7]].count(" ")

label = rand_data[8]
# ========================================
apply_chr = re.compile("[^가-힣]")
mask_num = 3

tokenized_sent = tokenizer.tokenize(sentence)
token_idx = []
word_idx = []
select_flag = []

chr_count = 0
for idx, token in enumerate(tokenized_sent):
    p_token = token.replace("#", "")
    
    token_idx.append(idx)
    
    word_si = chr_count
    word_ei = chr_count + len(p_token) - 1
    word_idx.append([word_si, word_ei])
    
    # select_flag : 0과 1로 구성, 1 값을 가지고 있는 토큰들 중 mask_num만큼 선택해 해당 tokenm을 [MASK]로 변환함.
    # 이 때 entity1과 관련 있는 토큰, entity2와 관련 있는 토큰 제외
    # 한글이 아닌 문자를 포함한 token 제외
    if word_si in range(e1_si, e1_ei + 1) or word_ei in range(e1_si, e1_ei + 1) or (word_si <= e1_si and word_ei >= e1_ei):
        select_flag.append(0)
    elif word_si in range(e2_si, e2_ei + 1) or word_ei in range(e2_si, e2_ei + 1) or (word_si <= e2_si and word_ei >= e2_ei):
        select_flag.append(0)
    elif apply_chr.search(token) is not None:
        select_flag.append(0)
    else:
        select_flag.append(1)
        
    chr_count += len(token)
    chr_count -= token.count("#")
    if chr_count < len(sentence) and sentence[chr_count] == " ":
        chr_count += 1

picks = [v for i, v in enumerate(token_idx) if select_flag[i] == 1]
picks = random.sample(picks, k=3)

print(f"Original Sentence : {sentence}\n")
for pick_idx in picks:
    word_si, word_ei = word_idx[pick_idx]
    masked_sentence = sentence[:word_si] + "[MASK]" + sentence[word_ei + 1:]
    print(f"Masked Sentence : {masked_sentence}")
#     print(mask_filler(masked_sentence))
    for predict_sent in mask_filler(masked_sentence):
        for k, v in predict_sent.items():
            print(f"{k} : {v}")
        print()

Original Sentence : 외식문화기업 원앤원㈜(대표이사 박천희)의 브랜드 ‘원할머니보쌈·족발’과 ‘박가부대’가 오는 11월 21일부터 23일까지 ㈜월드전람에서 주최하는 ‘제 53회 프랜차이즈 창업박람회 2019 SETEC’에 참석한다고 20일 밝혔다.

Masked Sentence : 외식문화기업 원앤원㈜(대표이사 박천희)의 브랜드 ‘원할머니보쌈·족발’과 ‘박가부대’가 오는 11월 21일부터 23일까지 ㈜월드전람에서 주최하는 ‘제 53회 프랜차이즈 [MASK]박람회 2019 SETEC’에 참석한다고 20일 밝혔다.
sequence : [CLS] 외식문화기업 원앤원㈜ ( 대표이사 박천희 ) 의 브랜드 ‘ 원할머니보쌈 · 족발 ’ 과 ‘ 박가부대 ’ 가 오는 11월 21일부터 23일까지 ㈜월드전람에서 주최하는 ‘ 제 53회 프랜차이즈 외식 박람회 2019 SETEC ’ 에 참석한다고 20일 밝혔다. [SEP]
score : 0.20184636116027832
token : 14153
token_str : 외식

sequence : [CLS] 외식문화기업 원앤원㈜ ( 대표이사 박천희 ) 의 브랜드 ‘ 원할머니보쌈 · 족발 ’ 과 ‘ 박가부대 ’ 가 오는 11월 21일부터 23일까지 ㈜월드전람에서 주최하는 ‘ 제 53회 프랜차이즈 프랜차이즈 박람회 2019 SETEC ’ 에 참석한다고 20일 밝혔다. [SEP]
score : 0.1871740221977234
token : 13508
token_str : 프랜차이즈

sequence : [CLS] 외식문화기업 원앤원㈜ ( 대표이사 박천희 ) 의 브랜드 ‘ 원할머니보쌈 · 족발 ’ 과 ‘ 박가부대 ’ 가 오는 11월 21일부터 23일까지 ㈜월드전람에서 주최하는 ‘ 제 53회 프랜차이즈 한식 박람회 2019 SETEC ’ 에 참석한다고 20일 밝혔다. [SEP]
score : 0.055190883576869965
token : 15532
token_str : 한식

sequence : [CLS] 외식문화기업 원

## 함수로 구현하기

In [ ]:
def text_augmentation(data, tokenizer, mask_filler):
    apply_chr = re.compile("[^a-zA-Z0-9가-힣]")
    
    source = data[0]
    sentence = data[1]
    
    e1_name = data[2]
    e1_si = data[3]
    e1_ei = data[4]
    
    e2_name = data[5]
    e2_si = data[6]
    e2_ei = data[7]
    
    label = data[8]
    
    nouns = [token for token in tokenizer(sentence) if apply_chr.search(token)]